In [1]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


SEARCH_WORD = "EXA FIRST"

DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "アスカ狭山店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    }

# 検索キーワードよりホール名取得
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
        print(f"🔍 '{SEARCH_WORD}' を含むホール名が見つかりました。")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals):
    bb_medals = 239.25
    rb_medals = 95.25
    cherry_rate_high = 0.04228
    cherry_rate_low = 0.05847
    replay_rate = 0.411
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"]).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
model_list = df["model_name"].unique()
print(f'以下のモデルが含まれています')
for i, model in enumerate(model_list):
    print(f'{i}: {model}')

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 57608 x 16
以下のモデルが含まれています
0: ゴーゴージャグラー3
1: マイジャグラーV
2: ファンキージャグラー2
3: ミスタージャグラー
4: アイムジャグラーEX-TP
5: ハッピージャグラーVIII
6: ウルトラミラクルジャグラー
7: ジャグラーガールズ
8: SミスタージャグラーKK


In [2]:
df.head()

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,grape_rate,Total_rate,month,day,weekday,unit_last
0,EXA FIRST,2025-04-17,ゴーゴージャグラー3,1001,5488,15,19,-1382,365.9,288.8,6.10,161.4,2025-04,17,3,1
1,EXA FIRST,2025-04-17,ゴーゴージャグラー3,1002,800,1,2,-626,800.0,400.0,6.52,266.7,2025-04,17,3,2
2,EXA FIRST,2025-04-17,ゴーゴージャグラー3,1003,6994,38,20,2197,184.1,349.7,6.21,120.6,2025-04,17,3,3
3,EXA FIRST,2025-04-17,ゴーゴージャグラー3,1004,1695,7,5,38,242.1,339.0,6.15,141.2,2025-04,17,3,4
4,EXA FIRST,2025-04-17,ゴーゴージャグラー3,1005,4432,19,17,432,233.3,260.7,6.38,123.1,2025-04,17,3,5


## RB_RATE

In [3]:
model_name = model_list[1]
today = datetime.date.today()
target_date = today - datetime.timedelta(days=1)
start = today - relativedelta(days=1)
end = today - relativedelta(days=61)
unit_no = 1021

df_tmp = df[(df["model_name"] == model_name)].copy()
df_tmp = df_tmp[(df_tmp["date"].dt.date <= start) & (df_tmp["date"].dt.date >= end)]
df_tmp = df_tmp[df_tmp["unit_no"] >= unit_no]

game = df_tmp.pivot_table(index=["model_name", "unit_no", ], columns="date", values="game", aggfunc="sum",)
bb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="BB", aggfunc="sum",)
rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="RB", aggfunc="sum",)
rb_rate = (game / rb).round(1)
total_rate = (game / (bb+rb)).round(1)

rb_rate[today] = ""
rb_rate = rb_rate.iloc[:, 7:].iloc[:, ::-1]

medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="medals", aggfunc="sum",)
rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
rolling_7d_sum.columns = [f"{col.strftime('%y-%m-%d')}_7d_sum" for col in rolling_7d_sum.columns]
rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]

C:\Users\mkyag\AppData\Local\Temp\ipykernel_32504\288722337.py:22: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]


In [4]:
print(rolling_7d_sum.shape)
print(rb_rate.shape)
display(rolling_7d_sum.head(3))
display(rb_rate.head(3))

(61, 53)
(61, 53)


25-04-11_7d_sum  25-04-10_7d_sum  25-04-09_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021             -2389.0          -1068.0          -1333.0   
           1022              2574.0           2248.0           2004.0   
           1023              -282.0           -988.0           -200.0   

                    25-04-08_7d_sum  25-04-07_7d_sum  25-04-06_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021             -3765.0          -1129.0           -667.0   
           1022              1406.0           1583.0            904.0   
           1023               944.0             38.0              2.0   

                    25-04-05_7d_sum  25-04-04_7d_sum  25-04-03_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021                24.0           1016.0            528.0   
           1022               118.0            827.0            262.0   
           1023               758.0           -598.0          -1548.0   

                    25-04-02_7d_sum  ...  25-02-25_7d_sum  25-02-24_7d_sum  \
model_name unit_no                   ...                                     
マイジャグラーV   1021               -42.0  ...           4468.0           3023.0   
           1022              -474.0  ...            318.0          -1285.0   
           1023             -1936.0  ...           1623.0           1929.0   

                    25-02-23_7d_sum  25-02-22_7d_sum  25-02-21_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021              3576.0           -583.0           -756.0   
           1022              -914.0           -179.0           1976.0   
           1023             -1601.0            308.0             90.0   

                    25-02-20_7d_sum  25-02-19_7d_sum  25-02-18_7d_sum  \
model_name unit_no                                                      
マイジャグラーV   1021             -2277.0          -2033.0          -2383.0   
           1022               541.0           1235.0           1985.0   
           1023              1075.0           1808.0            111.0   

                    25-02-17_7d_sum  25-02-16_7d_sum  
model_name unit_no                                    
マイジャグラーV   1021             -1550.0          -3012.0  
           1022               870.0           2720.0  
           1023              1091.0           2715.0  

[3 rows x 53 columns]

date               2025-04-18  2025-04-17 00:00:00  2025-04-16 00:00:00  \
model_name unit_no                                                        
マイジャグラーV   1021                              459.3                434.9   
           1022                              484.6                360.9   
           1023                              338.9               1230.0   

date                2025-04-15 00:00:00  2025-04-14 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   251.1                940.8   
           1022                   326.5                306.4   
           1023                   239.3                279.1   

date                2025-04-13 00:00:00  2025-04-12 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   659.8                375.3   
           1022                   613.0                271.6   
           1023                   448.5                329.8   

date                2025-04-11 00:00:00  2025-04-10 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   314.1                616.0   
           1022                   372.2                314.5   
           1023                   565.1                441.8   

date                2025-04-09 00:00:00  ...  2025-03-06 00:00:00  \
model_name unit_no                       ...                        
マイジャグラーV   1021                   435.4  ...                451.0   
           1022                   349.8  ...                333.7   
           1023                   420.8  ...                362.8   

date                2025-03-05 00:00:00  2025-03-04 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   416.2                324.2   
           1022                   336.0                627.0   
           1023                   393.0                286.5   

date                2025-03-03 00:00:00  2025-03-02 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   220.7                246.5   
           1022                   279.3                388.2   
           1023                   445.7                306.3   

date                2025-03-01 00:00:00  2025-02-26 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   288.1                549.0   
           1022                   301.6                342.1   
           1023                   478.3                475.0   

date                2025-02-25 00:00:00  2025-02-24 00:00:00  \
model_name unit_no                                             
マイジャグラーV   1021                   274.9                445.5   
           1022                   487.6                429.7   
           1023                   300.3                349.9   

date                2025-02-23 00:00:00  
model_name unit_no                       
マイジャグラーV   1021                   378.5  
           1022                   536.0  
           1023                   392.8  

[3 rows x 53 columns]

In [14]:
# 同じ列数を前提（もしくは短い方に合わせる）
columns_interleaved = [
    col for pair in zip(rb_rate.columns, rolling_7d_sum.columns) for col in pair
]
merged = pd.concat([rb_rate, rolling_7d_sum], axis=1)[columns_interleaved]
# 空行を追加
empty_index = pd.MultiIndex.from_tuples([(model_name, "")], names=merged.index.names)
merged_with_blanks = pd.DataFrame(
    [[""] * merged.shape[1]], columns=merged.columns, index=empty_index
)
empty_row = merged_with_blanks.copy()
blocks = [
    merged.iloc[0:6],
    merged.iloc[6:12],
    merged.iloc[12:21],
    merged.iloc[21:30],
    merged.iloc[30:46],
    merged.iloc[46:54],
    merged.iloc[54:61],
]
for block in blocks:
    merged_with_blanks = pd.concat([merged_with_blanks, block, empty_row])
merged_with_blanks.head()

2025-04-18 25-04-11_7d_sum 2025-04-17 00:00:00  \
model_name unit_no                                                  
マイジャグラーV                                                            
           1021                       -2389.0               459.3   
           1022                        2574.0               484.6   
           1023                        -282.0               338.9   
           1024                        4408.0               393.4   

                   25-04-10_7d_sum 2025-04-16 00:00:00 25-04-09_7d_sum  \
model_name unit_no                                                       
マイジャグラーV                                                                 
           1021            -1068.0               434.9         -1333.0   
           1022             2248.0               360.9          2004.0   
           1023             -988.0              1230.0          -200.0   
           1024             2129.0               466.6          3017.0   

                   2025-04-15 00:00:00 25-04-08_7d_sum 2025-04-14 00:00:00  \
model_name unit_no                                                           
マイジャグラーV                                                                     
           1021                  251.1         -3765.0               940.8   
           1022                  326.5          1406.0               306.4   
           1023                  239.3           944.0               279.1   
           1024                  322.6          3153.0               381.2   

                   25-04-07_7d_sum  ... 2025-03-01 00:00:00 25-02-20_7d_sum  \
model_name unit_no                  ...                                       
マイジャグラーV                            ...                                       
           1021            -1129.0  ...               288.1         -2277.0   
           1022             1583.0  ...               301.6           541.0   
           1023               38.0  ...               478.3          1075.0   
           1024             2315.0  ...               338.2         -1417.0   

                   2025-02-26 00:00:00 25-02-19_7d_sum 2025-02-25 00:00:00  \
model_name unit_no                                                           
マイジャグラーV                                                                     
           1021                  549.0         -2033.0               274.9   
           1022                  342.1          1235.0               487.6   
           1023                  475.0          1808.0               300.3   
           1024                  456.8          -687.0               327.7   

                   25-02-18_7d_sum 2025-02-24 00:00:00 25-02-17_7d_sum  \
model_name unit_no                                                       
マイジャグラーV                                                                 
           1021            -2383.0               445.5         -1550.0   
           1022             1985.0               429.7           870.0   
           1023              111.0               349.9          1091.0   
           1024            -1473.0               325.0          1980.0   

                   2025-02-23 00:00:00 25-02-16_7d_sum  
model_name unit_no                                      
マイジャグラーV                                                
           1021                  378.5         -3012.0  
           1022                  536.0          2720.0  
           1023                  392.8          2715.0  
           1024                  529.5          1966.0  

[5 rows x 106 columns]

In [15]:
merged_with_blanks.to_csv(f"{model_name}_merged.csv", encoding="utf_8_sig")